### Import Libraries and Read Data

In [430]:
## Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
#%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
plt.rcParams['figure.figsize'] = [15, 5]
from IPython import display
from ipywidgets import interact, widgets

## Read Data for Cases, Deaths and Recoveries
ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
Deaths_raw=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
Recoveries_raw=pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

In [431]:
ConfirmedCases_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
0,Anhui,Mainland China,31.8257,117.2264,1,9,15,39,60,70,...,989,989,989,990,990,990,990,990,990,990
1,Beijing,Mainland China,40.1824,116.4142,14,22,36,41,68,80,...,400,400,410,410,411,413,414,414,418,418
2,Chongqing,Mainland China,30.0572,107.8740,6,9,27,57,75,110,...,576,576,576,576,576,576,576,576,576,576
3,Fujian,Mainland China,26.0789,117.9874,1,5,10,18,35,59,...,294,294,296,296,296,296,296,296,296,296
4,Gansu,Mainland China,36.0611,103.8343,0,2,2,4,7,14,...,91,91,91,91,91,91,91,91,91,102


In [432]:
### Melt the dateframe into the right shape and set index
def cleandata(df_raw):
    df_cleaned=df_raw.melt(id_vars=['Province/State','Country/Region','Lat','Long'],value_name='Cases',var_name='Date')
    df_cleaned=df_cleaned.set_index(['Country/Region','Province/State','Date'])
    return df_cleaned 

### Get Countrywise Data
def countrydata(df_cleaned,oldname,newname):
    df_country=df_cleaned.groupby(['Country/Region','Date'])['Cases'].sum().reset_index()
    df_country=df_country.set_index(['Country/Region','Date'])
    df_country.index=df_country.index.set_levels([df_country.index.levels[0], pd.to_datetime(df_country.index.levels[1])])
    df_country=df_country.sort_values(['Country/Region','Date'],ascending=True)
    df_country=df_country.rename(columns={oldname:newname})
    return df_country

### Get DailyData from Cumulative sum
def dailydata(dfcountry,oldname,newname):
    dfcountrydaily=dfcountry.groupby(level=0).diff().fillna(0)
    dfcountrydaily=dfcountrydaily.rename(columns={oldname:newname})
    return dfcountrydaily

In [433]:
# Clean all datasets
ConfirmedCases=cleandata(ConfirmedCases_raw)
Deaths=cleandata(Deaths_raw)
Recoveries=cleandata(Recoveries_raw)

In [434]:
ConfirmedCases.head()

Lat      Long  Cases
Country/Region Province/State Date                             
Mainland China Anhui          1/22/20  31.8257  117.2264      1
               Beijing        1/22/20  40.1824  116.4142     14
               Chongqing      1/22/20  30.0572  107.8740      6
               Fujian         1/22/20  26.0789  117.9874      1
               Gansu          1/22/20  36.0611  103.8343      0

In [435]:
# Get countrywise data
ConfirmedCasesCountry=countrydata(ConfirmedCases,'Cases','Total Confirmed Cases')
DeathsCountry=countrydata(Deaths,'Cases','Total Deaths')
RecoveriesCountry=countrydata(Recoveries,'Cases','Total Recoveries')

In [436]:
# Get Daily Data
NewCasesCountry=dailydata(ConfirmedCasesCountry,'Total Confirmed Cases','Daily New Cases')
NewDeathsCountry=dailydata(DeathsCountry,'Total Deaths','Daily New Deaths')
NewRecoveriesCountry=dailydata(RecoveriesCountry,'Total Recoveries','Daily New Recoveries')

In [437]:
CountryConsolidated=pd.merge(ConfirmedCasesCountry,NewCasesCountry,how='left',left_index=True,right_index=True)
CountryConsolidated=pd.merge(CountryConsolidated,NewDeathsCountry,how='left',left_index=True,right_index=True)
CountryConsolidated=pd.merge(CountryConsolidated,DeathsCountry,how='left',left_index=True,right_index=True)
CountryConsolidated=pd.merge(CountryConsolidated,RecoveriesCountry,how='left',left_index=True,right_index=True)
CountryConsolidated=pd.merge(CountryConsolidated,NewRecoveriesCountry,how='left',left_index=True,right_index=True)
CountryConsolidated['Active Cases']=CountryConsolidated['Total Confirmed Cases']-CountryConsolidated['Total Deaths']-CountryConsolidated['Total Recoveries']
CountryConsolidated['Share of Recoveries - Closed Cases']=np.round(CountryConsolidated['Total Recoveries']/(CountryConsolidated['Total Recoveries']+CountryConsolidated['Total Deaths']),2)
CountryConsolidated['Death to Cases Ratio']=np.round(CountryConsolidated['Total Deaths']/CountryConsolidated['Total Confirmed Cases'],3)


In [438]:
CountryConsolidated.tail(4)

Total Confirmed Cases  Daily New Cases  \
Country/Region Date                                                 
Vietnam        2020-03-02                     16              0.0   
               2020-03-03                     16              0.0   
               2020-03-04                     16              0.0   
               2020-03-05                     16              0.0   

                           Daily New Deaths  Total Deaths  Total Recoveries  \
Country/Region Date                                                           
Vietnam        2020-03-02               0.0             0                16   
               2020-03-03               0.0             0                16   
               2020-03-04               0.0             0                16   
               2020-03-05               0.0             0                16   

                           Daily New Recoveries  Active Cases  \
Country/Region Date                                             
Vietnam        2020-03-02                   0.0             0   
               2020-03-03                   0.0             0   
               2020-03-04                   0.0             0   
               2020-03-05                   0.0             0   

                           Share of Recoveries - Closed Cases  \
Country/Region Date                                             
Vietnam        2020-03-02                                 1.0   
               2020-03-03                                 1.0   
               2020-03-04                                 1.0   
               2020-03-05                                 1.0   

                           Death to Cases Ratio  
Country/Region Date                              
Vietnam        2020-03-02                   0.0  
               2020-03-03                   0.0  
               2020-03-04                   0.0  
               2020-03-05                   0.0

In [439]:
GlobalTotals=CountryConsolidated.reset_index().groupby('Date').sum()
GlobalTotals['Share of Recoveries - Closed Cases']=np.round(GlobalTotals['Total Recoveries']/(GlobalTotals['Total Recoveries']+GlobalTotals['Total Deaths']),2)
GlobalTotals['Death to Cases Ratio']=np.round(GlobalTotals['Total Deaths']/GlobalTotals['Total Confirmed Cases'],3)
GlobalTotals.tail(2)

,Total Confirmed Cases,Daily New Cases,Daily New Deaths,Total Deaths,Total Recoveries,Daily New Recoveries,Active Cases,Share of Recoveries - Closed Cases,Death to Cases Ratio
Date,,,,,,,,,
2020-03-04,95124,2280.0,94.0,3254,51171,2942.0,40699,0.94,0.034
2020-03-05,97886,2762.0,94.0,3348,53797,2626.0,40741,0.94,0.034


###  Cases Worldwide by Status: Total Confirmed, Active, Deaths, Recoveries

In [440]:
GlobalTotals.iloc[-1]

Total Confirmed Cases                 97886.000
Daily New Cases                        2762.000
Daily New Deaths                         94.000
Total Deaths                           3348.000
Total Recoveries                      53797.000
Daily New Recoveries                   2626.000
Active Cases                          40741.000
Share of Recoveries - Closed Cases        0.940
Death to Cases Ratio                      0.034
Name: 2020-03-05 00:00:00, dtype: float64

In [441]:
plt.style.use('seaborn-dark')

In [442]:
chartcol='red'
fig = make_subplots(rows=3, cols=2,shared_xaxes=True,
                    specs=[[{}, {}],[{},{}],
                       [{"colspan": 2}, None]],
                    subplot_titles=('Total Confirmed Cases','Active Cases','Deaths','Recoveries','Death to Cases Ratio'))
fig.add_trace(go.Scatter(x=GlobalTotals.index,y=GlobalTotals['Total Confirmed Cases'],
                         mode='lines+markers',
                         name='Confirmed Cases',
                         line=dict(color=chartcol,width=2)),
                         row=1,col=1)
fig.add_trace(go.Scatter(x=GlobalTotals.index,y=GlobalTotals['Active Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                         row=1,col=2)
fig.add_trace(go.Scatter(x=GlobalTotals.index,y=GlobalTotals['Total Deaths'],
                         mode='lines+markers',
                         name='Deaths',
                         line=dict(color=chartcol,width=2)),
                         row=2,col=1)
fig.add_trace(go.Scatter(x=GlobalTotals.index,y=GlobalTotals['Total Recoveries'],
                         mode='lines+markers',
                         name='Recoveries',
                         line=dict(color=chartcol,width=2)),
                         row=2,col=2)
fig.add_trace(go.Scatter(x=GlobalTotals.index,y=GlobalTotals['Death to Cases Ratio'],
                         mode='lines+markers',
                         line=dict(color=chartcol,width=2)),
                         row=3,col=1)
fig.update_layout(showlegend=False)

### Cases by Region (Insite or Outside China)

In [443]:
#CountryConsolidated['location']=np.where(CountryConsolidated['Country/Region']=='Mainland China','Mainland China','Outside China')
locdata=CountryConsolidated.reset_index()
locdata['location']=np.where(locdata['Country/Region']=='Mainland China','Mainland China','Outside China')
locdata.head()

,Country/Region,Date,Total Confirmed Cases,Daily New Cases,Daily New Deaths,Total Deaths,Total Recoveries,Daily New Recoveries,Active Cases,Share of Recoveries - Closed Cases,Death to Cases Ratio,location
0,Afghanistan,2020-01-22,0,0.0,0.0,0,0,0.0,0,NaN,NaN,Outside China
1,Afghanistan,2020-01-23,0,0.0,0.0,0,0,0.0,0,NaN,NaN,Outside China
2,Afghanistan,2020-01-24,0,0.0,0.0,0,0,0.0,0,NaN,NaN,Outside China
3,Afghanistan,2020-01-25,0,0.0,0.0,0,0,0.0,0,NaN,NaN,Outside China
4,Afghanistan,2020-01-26,0,0.0,0.0,0,0,0.0,0,NaN,NaN,Outside China


In [444]:
# Cases by Region (inside or Outside China)
TotalCasesLoc=CountryConsolidated.reset_index()
TotalCasesLoc.head()
TotalCasesLoc['Location']=np.where(TotalCasesLoc['Country/Region']=='Mainland China','Mainland China','Outside China')
TotalCasesLocaggregated=TotalCasesLoc.groupby(['Location','Date'])['Active Cases'].sum().reset_index()
ActiveCasesChina=TotalCasesLocaggregated[TotalCasesLocaggregated.loc[:,'Location']=='Mainland China']
ActiveCasesExcludingChina=TotalCasesLocaggregated[TotalCasesLocaggregated.loc[:,'Location']=='Outside China']
ActiveCasesExcludingChina.head()

,Location,Date,Active Cases
44,Outside China,2020-01-22,8
45,Outside China,2020-01-23,14
46,Outside China,2020-01-24,25
47,Outside China,2020-01-25,35
48,Outside China,2020-01-26,53


In [445]:
fig = make_subplots(rows=1, cols=2,shared_xaxes=True,
                   subplot_titles=('Active Cases in Mainland China','Active Cases Excluding Mainland China'))
fig.add_trace(go.Scatter(x=ActiveCasesChina['Date'],y=ActiveCasesChina['Active Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                         row=1,col=1)
fig.add_trace(go.Scatter(x=ActiveCasesExcludingChina['Date'],y=ActiveCasesExcludingChina['Active Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                         row=1,col=2)
fig.update_layout(showlegend=False)

### Cases by Country

In [446]:
def plotcountry(Country):
    fig = make_subplots(rows=3, cols=2,shared_xaxes=True,
                    specs=[[{}, {}],[{},{}],
                       [{"colspan": 2}, None]],
                    subplot_titles=('Total Confirmed Cases','Active Cases','Deaths','Recoveries','Death to Cases Ratio'))
    fig.add_trace(go.Scatter(x=CountryConsolidated.loc[Country].index,y=CountryConsolidated.loc[Country,'Total Confirmed Cases'],
                         mode='lines+markers',
                         name='Confirmed Cases',
                         line=dict(color=chartcol,width=2)),
                         row=1,col=1)
    fig.add_trace(go.Scatter(x=CountryConsolidated.loc[Country].index,y=CountryConsolidated.loc[Country,'Active Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                         row=1,col=2)
    fig.add_trace(go.Scatter(x=CountryConsolidated.loc[Country].index,y=CountryConsolidated.loc[Country,'Total Deaths'],
                         mode='lines+markers',
                         name='Total Deaths',
                         line=dict(color=chartcol,width=2)),
                         row=2,col=1)
    fig.add_trace(go.Scatter(x=CountryConsolidated.loc[Country].index,y=CountryConsolidated.loc[Country,'Total Recoveries'],
                         mode='lines+markers',
                         name='Total Recoveries',
                         line=dict(color=chartcol,width=2)),
                         row=2,col=2)
    fig.add_trace(go.Scatter(x=CountryConsolidated.loc[Country].index,y=CountryConsolidated.loc[Country,'Death to Cases Ratio'],
                         mode='lines+markers',
                         name='Death to Cases Ratio',
                         line=dict(color=chartcol,width=2)),
                         row=3,col=1)
    fig.update_layout(showlegend=False)
    
    return fig
CountriesList=['Germany','Mainland China','US','Italy','France','Pakistan','South Korea','Japan','Singapore']
interact(plotcountry, Country=widgets.Dropdown(options=CountriesList))

interactive(children=(Dropdown(description='Country', options=('Germany', 'Mainland China', 'US', 'Italy', 'Fr…

<function __main__.plotcountry(Country)>

In [447]:
TotalCasesCountry=CountryConsolidated.max(level=0)['Total Confirmed Cases'].reset_index().set_index('Country/Region')
TotalCasesCountry=TotalCasesCountry.sort_values(by='Total Confirmed Cases',ascending=False)
TotalCasesCountryexclChina=TotalCasesCountry[~TotalCasesCountry.index.isin(['Mainland China','Others'])]
Top10countriesbycasesexclChina=TotalCasesCountryexclChina.head(10)
TotalCasesCountrytop10=TotalCasesCountry.head(10)
fig = go.Figure(go.Bar(x=Top10countriesbycasesexclChina.index, y=Top10countriesbycasesexclChina['Total Confirmed Cases'],
                      text=Top10countriesbycasesexclChina['Total Confirmed Cases'],
            textposition='outside'))
fig.update_layout(title_text='Top 10 Countries by Total Confirmed Cases Excluding China')
fig.update_yaxes(showticklabels=False)

fig.show()

### Total Cases by number of days since 'Outbreak'

In [448]:
ItalyFirstCase=CountryConsolidated.loc['Italy']['Total Confirmed Cases'].reset_index().set_index('Date')
SKFirstCase=CountryConsolidated.loc['South Korea']['Total Confirmed Cases'].reset_index().set_index('Date')
IranFirstCase=CountryConsolidated.loc['Iran']['Total Confirmed Cases'].reset_index().set_index('Date')
GermanyFirstCase=CountryConsolidated.loc['Germany']['Total Confirmed Cases'].reset_index().set_index('Date')
SingaporeFirstCase=CountryConsolidated.loc['Singapore']['Total Confirmed Cases'].reset_index().set_index('Date')

ItalyGrowth=ItalyFirstCase[ItalyFirstCase.ne(0)].dropna().reset_index()
SKGrowth=SKFirstCase[SKFirstCase.ne(0)].dropna().reset_index()
IranGrowth=IranFirstCase[IranFirstCase.ne(0)].dropna().reset_index()
GermanyGrowth=GermanyFirstCase[GermanyFirstCase.ne(0)].dropna().reset_index()
SingaporeGrowth=SingaporeFirstCase[SingaporeFirstCase.ne(0)].dropna().reset_index()

In [449]:
ItalyGrowth.head()

,Date,Total Confirmed Cases
0,2020-01-31,2.0
1,2020-02-01,2.0
2,2020-02-02,2.0
3,2020-02-03,2.0
4,2020-02-04,2.0


In [450]:
fig = make_subplots(rows=2, cols=2,shared_xaxes=True,
                   subplot_titles=('Italy','Iran','South Korea','Germany'))

fig.update_xaxes(title_text="Number of Days since Outbreak", row=2, col=1)

fig.update_xaxes(title_text="Number of Days since Outbreak", row=2, col=2)

fig.add_trace(go.Scatter(x=ItalyGrowth.index,y=ItalyGrowth['Total Confirmed Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                          row=1,col=1)
fig.add_trace(go.Scatter(x=IranGrowth.index,y=IranGrowth['Total Confirmed Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                          row=1,col=2)
                           
fig.add_trace(go.Scatter(x=SKGrowth.index,y=SKGrowth['Total Confirmed Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                          row=2,col=1)    
                           
fig.add_trace(go.Scatter(x=GermanyGrowth.index,y=GermanyGrowth['Total Confirmed Cases'],
                         mode='lines+markers',
                         name='Active Cases',
                         line=dict(color=chartcol,width=2)),
                          row=2,col=2)  

fig.update_layout(showlegend=False)